In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from collections import OrderedDict
from tqdm import tqdm
import datetime

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
dataset = datasets.ImageFolder('D:/data2', transform=transforms.Compose([
        transforms.Resize((644, 644)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))
class_names = dataset.classes
len(class_names)

7

In [3]:
trainds, valds, testds = torch.utils.data.random_split(dataset, [0.4, 0.3, 0.3], generator=torch.Generator().manual_seed(15275))

dataloaders = {'train': torch.utils.data.DataLoader(trainds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'val': torch.utils.data.DataLoader(valds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'test': torch.utils.data.DataLoader(testds, batch_size=32,
                                             shuffle=True, num_workers=16)}

dataset_sizes = {'train': len(trainds), 
               'val': len(valds), 
               'test': len(testds)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
len(class_names)

7

In [8]:
# def imshow(inp, title=None):
#     """Display image for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []


    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)
            print(datetime.datetime.now())

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tqdm(dataloaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]
                if phase == 'train':
                    train_loss.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    valid_loss.append(epoch_loss)
                    valid_acc.append(epoch_acc)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model, train_loss, train_acc, valid_loss, valid_acc

In [10]:
class SKAttention(nn.Module):
    def __init__(self, channel=512, kernels=[1, 3, 5, 7], reduction=16, group=1, L=32):
        super().__init__()
        self.d = max(L, channel // reduction)
        self.convs = nn.ModuleList([])
        for k in kernels:
            self.convs.append(
                nn.Sequential(OrderedDict([
                    ('conv', nn.Conv2d(channel, channel, kernel_size=k, padding=k // 2, groups=group)),
                    ('bn', nn.BatchNorm2d(channel)),
                    ('relu', nn.ReLU())
                ]))
            )
        self.fc = nn.Linear(channel, self.d)
        self.fcs = nn.ModuleList([])
        for i in range(len(kernels)):
            self.fcs.append(nn.Linear(self.d, channel))
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        bs, c, _, _ = x.size()
        conv_outs = []
        ### split
        for conv in self.convs:
            conv_outs.append(conv(x))
        feats = torch.stack(conv_outs, 0)  # k,bs,channel,h,w

        ### fuse
        U = sum(conv_outs)  # bs,c,h,w

        ### reduction channel
        S = U.mean(-1).mean(-1)  # bs,c
        Z = self.fc(S)  # bs,d

        ### calculate attention weight
        weights = []
        for fc in self.fcs:
            weight = fc(Z)
            weights.append(weight.view(bs, c, 1, 1))  # bs,channel
        attention_weughts = torch.stack(weights, 0)  # k,bs,channel,1,1
        attention_weughts = self.softmax(attention_weughts)  # k,bs,channel,1,1

        ### fuse
        V = (attention_weughts * feats).sum(0)
        return V

In [11]:
model_ft = models.mobilenet_v3_large(weights='IMAGENET1K_V2', pretrained=True)
num_ftrs = model_ft.classifier[3].out_features
# Here the size of each output sample is set to 5.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.002)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [12]:
model_ft.features[14].block[2] = SKAttention(channel=960, reduction=8)
model_ft.features[15].block[2] = SKAttention(channel=960, reduction=8)

In [16]:
for name, param in model_ft.named_parameters():
    if param.requires_grad:
        print(f"{name} will be updated.")
    else:
        print(f"{name} will not be updated.")

features.0.0.weight will be updated.
features.0.1.weight will be updated.
features.0.1.bias will be updated.
features.1.block.0.0.weight will be updated.
features.1.block.0.1.weight will be updated.
features.1.block.0.1.bias will be updated.
features.1.block.1.0.weight will be updated.
features.1.block.1.1.weight will be updated.
features.1.block.1.1.bias will be updated.
features.2.block.0.0.weight will be updated.
features.2.block.0.1.weight will be updated.
features.2.block.0.1.bias will be updated.
features.2.block.1.0.weight will be updated.
features.2.block.1.1.weight will be updated.
features.2.block.1.1.bias will be updated.
features.2.block.2.0.weight will be updated.
features.2.block.2.1.weight will be updated.
features.2.block.2.1.bias will be updated.
features.3.block.0.0.weight will be updated.
features.3.block.0.1.weight will be updated.
features.3.block.0.1.bias will be updated.
features.3.block.1.0.weight will be updated.
features.3.block.1.1.weight will be updated.
fea

In [17]:
model_ft, train_loss, train_acc, valid_loss, valid_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=70)

Epoch 0/69
----------
2025-03-05 21:15:19.973638


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:39<00:00, 39.87s/it]


train Loss: 3.4740 Acc: 0.3966


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:45<00:00, 15.13s/it]


val Loss: 4.2016 Acc: 0.0698

Epoch 1/69
----------
2025-03-05 21:18:49.868963


...

val Loss: 0.0430 Acc: 0.9767

Epoch 69/69
----------
2025-03-06 01:09:58.813100


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:45<00:00, 41.33s/it]


train Loss: 0.0129 Acc: 1.0000


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:46<00:00, 15.39s/it]


val Loss: 0.0416 Acc: 0.9884

Training complete in 238m 21s
Best val Acc: 0.988372


In [18]:
def test_model(model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(testds)
    correct = 0.0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloaders['test']:
            X= X.to(device)
            outputs = model(X)
            _, preds = torch.max(outputs, 1)
            # print('===================')
            # print(preds)
            # print(y.data)
            correct += torch.sum(preds == y.data)

    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%")

test_model(model_ft, criterion)

Test Error: 
 Accuracy: 95.3%
